In [14]:
from keras.datasets import mnist
from matplotlib import pyplot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Conv2DTranspose, UpSampling2D
import numpy as np
from keras.constraints import max_norm
from sklearn.model_selection import train_test_split

In [15]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()


In [16]:
noise_factor = 0.25
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)
x_train_noisy = np.clip(x_train_noisy, 0. , 1.)
x_test_noisy = np.clip(x_test_noisy, 0. , 1.)

In [17]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train_noisy = x_train_noisy.reshape(x_train_noisy.shape[0], 28, 28, 1)
x_test_noisy = x_test_noisy.reshape(x_test_noisy.shape[0], 28, 28, 1)

In [18]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train_noisy = x_train_noisy.astype('float32')
x_test_noisy = x_test_noisy.astype('float32')

In [19]:
x_train /= 255
x_test /= 255
x_train_noisy /= 255
x_test_noisy /= 255

In [20]:
# Convolutional Autoencoder Neural Network

# Encoding the image
auto_encoder = Sequential()
auto_encoder.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(28,28,1)))
auto_encoder.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
auto_encoder.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
auto_encoder.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

# Reconstructing the image
auto_encoder.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
auto_encoder.add(UpSampling2D((2, 2)))
auto_encoder.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
auto_encoder.add(UpSampling2D((2, 2)))
auto_encoder.add(Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same'))

In [21]:
auto_encoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
auto_encoder.fit(x_train_noisy, x_train,epochs=5)

Epoch 1/5
1875/1875 [==============================] - 56s 30ms/step - loss: 0.1597 - accuracy: 0.8087
Epoch 2/5
1875/1875 [==============================] - 57s 30ms/step - loss: 0.0955 - accuracy: 0.8135
Epoch 3/5
1875/1875 [==============================] - 57s 30ms/step - loss: 0.0899 - accuracy: 0.8142
Epoch 4/5
1875/1875 [==============================] - 60s 32ms/step - loss: 0.0873 - accuracy: 0.8145
Epoch 5/5
1875/1875 [==============================] - 59s 31ms/step - loss: 0.0858 - accuracy: 0.8146


In [22]:
# Removing the noice through auto encoding of the noisy data

x_train_noice_reduced = auto_encoder.predict(x_train_noisy)
x_test_noice_reduced = auto_encoder.predict(x_test_noisy)

In [23]:
# Model used in 1 and 2
model = Sequential()
model.add(Conv2D(28, activation='relu', kernel_size=(3,3), input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(50,activation='relu'))
model.add(Dense(10,activation='softmax'))
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.fit(x=x_train_noice_reduced,y=y_train, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.2277 - accuracy: 0.9311
Epoch 2/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0932 - accuracy: 0.9707
Epoch 3/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0722 - accuracy: 0.9769
Epoch 4/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0596 - accuracy: 0.9807
Epoch 5/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0504 - accuracy: 0.9837
Epoch 6/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0434 - accuracy: 0.9859
Epoch 7/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0359 - accuracy: 0.9880
Epoch 8/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0309 - accuracy: 0.9896
Epoch 9/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0269 - accuracy: 0.9914
Epoch 10/10
1875/1875 [==============================] - 12s 6ms/step - l

In [24]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.0633 - accuracy: 0.9824


[0.06326256692409515, 0.9824000000953674]